In [ ]:
# Cargar el DataFrame
df = pd.read_csv("../data/Airbnb_Open_Data.csv", low_memory=False, encoding='ISO-8859-1')

# Función para identificar valores no ASCII
def reemplazar_no_ascii(valor):
    if isinstance(valor, str) and not valor.isascii():
        return "999999"
    return valor

# Función para limpiar caracteres especiales en texto
def limpiar_texto(texto):
    if isinstance(texto, str) and not re.match(r'^[A-Za-z0-9 .,!?]+$', texto):
        return "999999"
    return texto

# Función para validar datos numéricos
def validar_numericos(valor):
    try:
        num = float(valor)
        if num < 0 or num > 99999999:  # Puedes ajustar este umbral según el caso
            return 999999
    except ValueError:
        return 999999  # Si no es un número, lo marcamos como inconsistente
    return num

# Aplicar validaciones en todas las columnas según su tipo
for col in df.columns:
    if df[col].dtype == "object":  # Columnas de texto
        df[col] = df[col].map(reemplazar_no_ascii)
        df[col] = df[col].map(limpiar_texto)
    else:  # Columnas numéricas
        df[col] = df[col].map(validar_numericos)

# Reemplazar NaN e infinitos con 999999
df = df.fillna(999999)
df.replace([float('inf'), float('-inf')], 999999, inplace=True)

# Mostrar primeras filas
print(df.head())